### Homework 2

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('hmelq.mini.csv')
#data=pd.read_csv('https://raw.githubusercontent.com/caglarmert/UVBMOB/master/titanic_train.csv') # veriseti yoksa internetten çekmek için


In [3]:
data.head()

,bad,loan,mortdue,value,reason,job,yoj,derog,delinq,clage,ninq,clno,debtinc
0,0,81200,18834,108355,HomeImp,NaN,28.0,0.0,0.0,139.14,0.0,14.0,34.042
1,0,12600,103960,127384,DebtCon,NaN,2.0,0.0,0.0,129.02,0.0,25.0,34.479
2,0,18000,46865,61266,DebtCon,NaN,5.0,0.0,0.0,102.59,2.0,9.0,26.354
3,0,10300,57676,71027,DebtCon,NaN,19.0,0.0,0.0,157.52,1.0,11.0,33.992
4,0,9400,56508,78358,DebtCon,NaN,17.0,0.0,0.0,141.93,0.0,11.0,32.327


In [9]:
stats={ #'count': pd.DataFrame.count,
        'sum': pd.DataFrame.sum,
        'mean': pd.DataFrame.mean,
        #'median': pd.DataFrame.meadian,
        #'mode' : pd.DataFrame.mode,
        'std': pd.DataFrame.std,
        'min': pd.DataFrame.min,
        'max': pd.DataFrame.max,
        'abs': pd.DataFrame.abs,
        'prod': pd.DataFrame.prod,
        'cumsum': pd.DataFrame.cumsum,
        'cumprod': pd.DataFrame.cumprod}

columns = data.columns.drop(['reason'])

for j in range (len(stats)):
    print(" ")
    print(str(list(stats.keys())[j]).upper()+str(" : "))
    print(" ")
    for i in columns:
        print(str(i) + "  " + str(list(stats.values())[j](data[i])))




 
SUM : 
 
bad  0
loan  1981100
mortdue  6218529
value  8807114
job  0.0
yoj  775.3
derog  7.0
delinq  8.0
clage  8416.080000000002
ninq  43.0
clno  982.0
debtinc  2933.7239999999997
 
MEAN : 
 
bad  0.0
loan  20011.11111111111
mortdue  62813.42424242424
value  88960.74747474748
job  nan
yoj  9.69125
derog  0.12962962962962962
delinq  0.13114754098360656
clage  137.96852459016395
ninq  0.7962962962962963
clno  16.098360655737704
debtinc  32.59693333333333
 
STD : 
 
bad  0.0
loan  19988.18982365096
mortdue  27561.845388622747
value  26648.955191198755
job  nan
yoj  7.739479486471876
derog  0.3390495203045245
delinq  0.34036303344446667
clage  43.00923047579692
ninq  1.0881805765385795
clno  6.0434673216423995
debtinc  5.066861907631907
 
MIN : 
 
bad  0
loan  5000
mortdue  12340
value  32220
job  nan
yoj  2.0
derog  0.0
delinq  0.0
clage  63.12
ninq  0.0
clno  9.0
debtinc  20.028
 
MAX : 
 
bad  0
loan  82200
mortdue  119879
value  147991
job  nan
yoj  28.0
derog  1.0
delinq  1.0
clage